### Imports

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import trainer_lib as tl

torch.manual_seed(2412231200)
random.seed(3009231410)
np.random.seed(2909231846)
np_random_state = np.random.RandomState(131002)

### Load data

In [2]:
df: pd.DataFrame = tl.load_country_wide_dataset('../data/country_data.csv')

X = df['el_load'].to_numpy(dtype=np.float32)
y = X.copy()

### Define models

In [3]:
class ConvNetSmall(nn.Module):
    def __init__(self, dropout=0.5, **kwargs):
        super(ConvNetSmall, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, 4, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 8, 4, padding=2),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(1, -1),
            nn.Dropout(dropout),
            nn.Linear(6 * 8, 3),
        )

    def forward(self, x):
        x = x.reshape(-1, 1, 24)
        x = self.conv(x)
        x = self.fc(x)
        return x
    
class ConvNetMedium(nn.Module):
    def __init__(self, dropout=0.5, **kwargs):
        super(ConvNetMedium, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, 4, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16, 32, 4, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 16, 4, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(1, -1),
            nn.Dropout(dropout),
            nn.Linear(3 * 16, 3),
        )

    def forward(self, x):
        x = x.reshape(-1, 1, 24)
        x = self.conv(x)
        x = self.fc(x)
        return x

### Grid search

In [7]:
grid = tl.Grid({
    'epochs': [200],  # we use early stop anyway
    'batch_size': [64], # so we have less params to search, this can be a single test afterward
    'lr': [0.001, 0.0001],
    'model': [ConvNetSmall, ConvNetMedium],
    'dropout': [0.5, 0.3],
    'n_splits': [6],
})

wrapper = tl.MIMOTSWrapper(ConvNetSmall(), seq_len=24, pred_len=3)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=3)
print(f"Best params: {b_p}\nBest score: {b_s}")

[Grid search 001] BEGIN - params: {'epochs': 200, 'batch_size': 64, 'lr': 0.001, 'model': <class '__main__.ConvNetSmall'>, 'dropout': 0.5, 'n_splits': 6}
[Fold 1] BEGIN - END - Metric loss: 74036.15625000
[Fold 2] BEGIN - END - Metric loss: 29839.77929688
[Fold 3] BEGIN - END - Metric loss: 28764.83203125
[Fold 4] BEGIN - END - Metric loss: 61825.18750000
[Fold 5] BEGIN - END - Metric loss: 82268.16406250
[Fold 6] BEGIN - END - Metric loss: 74487.77343750
[Grid search 001] END - Score: 58536.98209635 *
[Grid search 002] BEGIN - params: {'epochs': 200, 'batch_size': 64, 'lr': 0.0001, 'model': <class '__main__.ConvNetSmall'>, 'dropout': 0.5, 'n_splits': 6}
[Fold 1] BEGIN - END - Metric loss: 67067.86718750
[Fold 2] BEGIN - END - Metric loss: 47872.58984375
[Fold 3] BEGIN - END - Metric loss: 33076.72656250
[Fold 4] BEGIN - END - Metric loss: 81323.43750000
[Fold 5] BEGIN - END - Metric loss: 86466.56250000
[Fold 6] BEGIN - END - Metric loss: 80678.94531250
[Grid search 002] END - Score: 